# Env

In [4]:
# !pip install -U -q python-dotenv
# !pip install -U -q langchain
# !pip install -U -q langchain_openai
# !pip install -U -q openai
# !pip install -U -q huggingface_hub
# !pip install -q langchain_community
# !pip install -U -q langchain-text-splitters
# !pip install -U -q chromadb

import os

In [5]:
os.listdir()

['ollamaXlangchain.ipynb',
 '.DS_Store',
 'AllKipAudio',
 'ChromaDB',
 'env',
 '芋圓AI台語字典.key',
 '芋圓AI台語字典.pdf',
 'TaroAI',
 'Demo.mp4',
 'KIPTaigiSutian_All.csv']

In [ ]:
from dotenv import dotenv_values
import os

env = dotenv_values("env")

os.environ["HUGGINGFACEHUB_API_TOKEN"] = env['HUGGINGFACEHUB_API_TOKEN_2']

# 失敗的話會列印出：'沒讀到 HUGGINGFACEHUB_API key'
os.environ["HUGGINGFACEHUB_API_TOKEN"]

# Embeddings

In [5]:
import pandas as pd
df = pd.read_csv("./KIPTaigiSutian_All.csv", index_col=0).fillna("")
df.head(3)

,WordID,WordType,Hanji,KipLomaji,Category,OtherPronunciation,Synonyms,Antonyms,SenseID,POS,Sense,SentenceID,SentenceHanji,SentenceKipLomaji,SentenceMandarin
0,1,主詞目,一,tsi̍t,"性質、程度,數詞、量詞",,"2056-全,7844-規",,1,數詞,數目。,45549,一蕊花,tsi̍t luí hue,一朵花
1,1,主詞目,一,tsi̍t,"性質、程度,數詞、量詞",,"2056-全,7844-規",,2,形容詞,全部的、整個的。,45550,紅嬰仔哭甲一身軀汗。,Âng-enn-á khàu kah tsi̍t sin-khu kuānn.,小嬰兒哭得滿身大汗。
2,1,主詞目,一,tsi̍t,"性質、程度,數詞、量詞",,"2056-全,7844-規",,3,副詞,表示時間短、數量少、程度輕等。,61001,一睏仔,tsi̍t-khùn-á,一下子


In [7]:
column_names = {
    "WordID": "台語詞ID",
    "Hanji": "台語詞",
    "KipLomaji": "台語詞羅馬字",
    "POS": "詞性",
    "SenseID": "詞義ID",
    "Sense": "詞義",
    "SentenceID": "例句ID",
    "SentenceHanji": "例句台語漢字",
    "SentenceKipLomaji": "例句羅馬字",
    "SentenceMandarin": "例句華語翻譯",
}
cols = list(column_names.keys())
df['Joined'] = df[cols].apply(lambda row: '|'.join(map(str, row)), axis=1)
entries = []
for index, row in df.iterrows():
    content = ""
    for col in cols:
        content += f"{column_names[col]}: {row[col]}|"
    entries.append(content)
    # entries.append(str(index) + "|" + row["Joined"])
entries[:5]

['台語詞ID: 1|台語詞: 一|台語詞羅馬字: tsi̍t|詞性: 數詞|詞義ID: 1|詞義: 數目。|例句ID: 45549|例句台語漢字: 一蕊花|例句羅馬字: tsi̍t luí hue|例句華語翻譯: 一朵花|',
 '台語詞ID: 1|台語詞: 一|台語詞羅馬字: tsi̍t|詞性: 形容詞|詞義ID: 2|詞義: 全部的、整個的。|例句ID: 45550|例句台語漢字: 紅嬰仔哭甲一身軀汗。|例句羅馬字: Âng-enn-á khàu kah tsi̍t sin-khu kuānn.|例句華語翻譯: 小嬰兒哭得滿身大汗。|',
 '台語詞ID: 1|台語詞: 一|台語詞羅馬字: tsi̍t|詞性: 副詞|詞義ID: 3|詞義: 表示時間短、數量少、程度輕等。|例句ID: 61001|例句台語漢字: 一睏仔|例句羅馬字: tsi̍t-khùn-á|例句華語翻譯: 一下子|',
 '台語詞ID: 1|台語詞: 一|台語詞羅馬字: tsi̍t|詞性: 副詞|詞義ID: 3|詞義: 表示時間短、數量少、程度輕等。|例句ID: 61002|例句台語漢字: 一絲仔|例句羅馬字: tsi̍t-si-á|例句華語翻譯: 一點點|',
 '台語詞ID: 2|台語詞: 一|台語詞羅馬字: it|詞性: 數詞|詞義ID: 4|詞義: 數字。|例句ID: 0|例句台語漢字: |例句羅馬字: |例句華語翻譯: |']

In [9]:
# Get the embeddings engine ready
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

my_embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name= "aspire/acge_text_embedding", #"sentence-transformers/all-MiniLM-l6-v2",
    api_key=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)

In [10]:
# The vectorstore we use: Chroma
from langchain_community.vectorstores import Chroma
my_vectorstore = Chroma(embedding_function = my_embeddings,
                        persist_directory = './ChromaDB')

In [11]:
%time
chunk_size = 100
total_items = len(entries) - 34300
print(f"There are {total_items} entries. There will be {total_items / chunk_size} chunks")
for i in range(0, total_items, chunk_size):
    i = i + 34300
    print(f"Processing from {i}th item")
    chunk = entries[i:i+chunk_size]
    # Embed each row and add it to my vectorstore
    my_vectorstore.add_texts(chunk)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.77 µs
There are 897 entries. There will be 8.97 chunks
Processing from 34300th item


KeyboardInterrupt: 

# Retreiver

In [20]:
# Get the embeddings engine ready
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

my_embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name= "aspire/acge_text_embedding", #"sentence-transformers/all-MiniLM-l6-v2",
    api_key=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)
# The vectorstore we use: Chroma
from langchain_community.vectorstores import Chroma
my_vectorstore = Chroma(embedding_function = my_embeddings,
                        persist_directory = './ChromaDB')

# 定義 retriever
# 要求他只返回最相似(最可能和query相關) 的 2 個 document
retriever = my_vectorstore.as_retriever(search_kwargs={"k": 5})

In [13]:
# 測試用 retriever 搜尋看看
retriever.invoke('關於天氣的句子')

[Document(page_content='台語詞ID: 1028|台語詞: 天氣|台語詞羅馬字: thinn-khì|詞性: 名詞|詞義ID: 1421|詞義: 天候。在一定的區域和時間內的氣溫、溼度、氣壓、風向和雨量等狀況。|例句ID: 46619|例句台語漢字: 今仔日天氣無好，你出門的時愛會記得紮雨幔。|例句羅馬字: Kin-á-ji̍t thinn-khì bô hó, lí tshut-mn̂g ê sî ài ē-kì-tit tsah hōo-mua.|例句華語翻譯: 今天天氣不好，你出門的時候要記得帶雨衣。|'),
 Document(page_content='台語詞ID: 784|台語詞: 天|台語詞羅馬字: thinn|詞性: 名詞|詞義ID: 1069|詞義: 氣候。|例句ID: 46374|例句台語漢字: 落雨天|例句羅馬字: lo̍h-hōo-thinn|例句華語翻譯: 下雨天|'),
 Document(page_content='台語詞ID: 6152|台語詞: 氣|台語詞羅馬字: khuì|詞性: 名詞|詞義ID: 8614|詞義: 自然界陰晴、冷暖的現象。|例句ID: 52122|例句台語漢字: 節氣|例句羅馬字: tseh-khuì|例句華語翻譯: 二十四節氣|'),
 Document(page_content='台語詞ID: 2296|台語詞: 好天|台語詞羅馬字: hó-thinn|詞性: 名詞|詞義ID: 3197|詞義: 好天氣。晴朗、氣溫宜人，令人覺得舒爽。|例句ID: 47892|例句台語漢字: 好天的時阮欲來去𬦰山。|例句羅馬字: Hó-thinn ê sî guán beh lâi-khì peh-suann.|例句華語翻譯: 天氣好的時候我們要去爬山。|'),
 Document(page_content='台語詞ID: 4549|台語詞: 雨來天|台語詞羅馬字: hōo-lâi-thinn|詞性: 名詞|詞義ID: 6418|詞義: 雨天、下雨天。|例句ID: 0|例句台語漢字: |例句羅馬字: |例句華語翻譯: |')]

In [18]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

## 定義 prompt
template = """根據給定 context 回答問題:

以下是台語字典的資料，包括台語詞、該詞的台語羅馬拼音、該詞的意思、例句、例句的羅馬拼音、例句的中文翻譯：
{context}

Question: {question}

給出回應的時候請包含台語詞、該詞的羅馬拼音、以及釋義
"""
prompt = ChatPromptTemplate.from_template(template)


## 定義模型
# llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2",
#                           max_length=128, temperature=0.5,)
llm = Ollama(model="llama3")


## 多加了一個整理 splitted doc 的小函數
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


## 定義 chain
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
## 定義使用者問題 ＆ invoke chain
user_query = "旅行的台語是什麼？"
result = chain.invoke(user_query)
print(result)

According to the given context, the answer is:

台語詞: 旅行|台語詞羅馬字: lí-hîng/lú-hîng|

The meaning of "旅行" (lí-hîng/lú-hîng) in Taiwanese is "to travel or take a trip", which means to go on a journey, whether it's for leisure or business.
